In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor as rfr

In [ ]:
pip install -U pymoo

In [ ]:
#Evaluaate Performance
def test(model1, X_test, y_test):
  print(model1)
  print("")
  r1 = model1.predict(X_test)
  print("prediction 1:\n", r1)

  print("")
  #Score:
  print("Score for prediction")
  print("R square score:", r2_score(y_test, r1))
  print("Max error:", max_error(y_test, r1))
  print("")
  print("-------------------------------------")

In [ ]:
rs = 5138

In [ ]:
data = pd.read_csv('/content/TestData.csv')
data.head(2)

,CaO,SiO,Na2O,K2O,H20,FA,CA,Total Binder,DRCM,Strength
0,223.77,206.53,0.77,7.86,196.14,725.35,1110.16,560.40,1.10,45.7
1,174.04,160.64,0.60,6.11,196.14,692.93,1110.16,435.87,1.42,40.9


In [ ]:
def fraction_df(data):
  datfr = pd.DataFrame()
  datfr['C/B'] = data['CaO']/data['Total Binder']
  datfr['S/B'] = data['SiO']/data['Total Binder']
  datfr['H/B'] = data['H20']/data['Total Binder']
  datfr['A/B'] = (data['Na2O']+data['K2O'])/data['Total Binder']
  datfr['F/T'] = data['FA']/(data['FA']+data['CA'])
  datfr['C/T'] = data['CA']/(data['FA']+data['CA'])

  return datfr

In [ ]:
def f_val_min_max(data, features:list, change_pc=0):
  '''function to get the minimum and maximum value of a feature'''
  n_f = len(features)
  min_reduce = 1-change_pc/100
  max_increase = 1+change_pc/100
  min_vals = []
  max_vals = []
  for i in range(n_f):
    min_vals.append(min_reduce*min(data[features[i]]))
    max_vals.append(max_increase*max(data[features[i]]))

  return min_vals, max_vals

In [ ]:
dat = fraction_df(data)
datfr = pd.concat([dat, data['DRCM'], data['Strength']], axis = 1)

In [ ]:
data1 = datfr[['C/B', 'S/B', 'H/B', 'A/B', 'F/T', 'C/T', 'DRCM']]
data2 = datfr[['C/B', 'S/B', 'H/B', 'A/B', 'F/T', 'C/T','Strength']]
features = ['CaO', 'SiO', 'Na2O', 'K2O', 'H20', 'FA', 'CA', 'Total Binder']
n_f = len(features)

In [ ]:
#Diffusion Coefficient Split
output_d = ['DRCM']
Input_d = ['C/B', 'S/B', 'H/B', 'A/B', 'F/T', 'C/T']

Xd = data1[Input_d]
yd = data1[output_d]

D_train, D_test, yd_train, yd_test = train_test_split(Xd, yd, random_state = rs)

In [ ]:
#Diffusion Coefficient Train
rfr_d = rfr(max_depth = 12)#, min_samples_split=4, n_estimators=66, random_state=rs)
rfr_d.fit(D_train, yd_train)

test(rfr_d, D_test, yd_test)

r = rfr_d.predict(D_test)
s = np.array(yd_test["DRCM"])
l = []
for i in r:
  l.append(float(i))

abs(s - np.array(l))/s*100

RandomForestRegressor(max_depth=12)

prediction 1:
 [ 2.58939897  6.4095      3.3017      5.0261     12.7179      4.25151143
  6.6065     13.6868      7.3779      5.04955     1.8546      2.53199754
  6.91855     3.59975     5.9186      4.9762      1.8068      1.7545
  6.5588     12.9861      5.8157      7.8892      5.9945     10.766
  3.0666      4.60345     7.1791     10.5297    ]

Score for prediction
R square score: 0.7260780491999724
Max error: 5.9639999999999915

-------------------------------------


<ipython-input-429-5c1575e6a04d>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr_d.fit(D_train, yd_train)


array([ 4.83396633, 20.96794081, 32.75560081, 10.70704846, 47.53944316,
       52.39068949, 19.68297101,  1.53412463,  5.39857143,  2.42494929,
       13.7791411 ,  0.07895414, 53.06526549,  2.70945946, 27.55691554,
        8.17826087,  1.26775956, 16.19205298, 19.91697192,  3.16107383,
       14.09601182, 24.8647619 , 10.66318927, 35.64853556, 12.38285714,
       17.64847943, 32.45571956, 19.43611324])

In [ ]:
#Strength Split
output_s = ['Strength']
Input_s = ['C/B', 'S/B', 'H/B', 'A/B', 'F/T', 'C/T']

Xs = data2[Input_s]
ys = data2[output_s]

S_train, S_test, ys_train, ys_test = train_test_split(Xs, ys, random_state = rs)

In [ ]:
#Strength Train
rfr_s = rfr(max_depth = 30)#12, n_estimators=106, min_samples_split=4, random_state=rs)
rfr_s.fit(S_train, ys_train)

test(rfr_s, S_test, ys_test)

r = rfr_s.predict(S_test)
s = np.array(ys_test['Strength'])
l = []
for i in r:
  l.append(float(i))

abs(s - np.array(l))/s*100

<ipython-input-431-3d6495b4d057>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr_s.fit(S_train, ys_train)


RandomForestRegressor(max_depth=30)

prediction 1:
 [47.842  45.5163 34.8758 51.6549 27.8702 45.9503 34.4838 25.6769 35.2095
 50.2609 38.0134 46.5821 34.5635 49.6682 45.5163 51.8547 37.5926 37.3985
 34.0789 21.3344 45.8268 22.5946 45.8268 24.6801 36.3865 36.0276 36.9168
 34.3304]

Score for prediction
R square score: 0.7558768525434861
Max error: 13.336500000000001

-------------------------------------


array([7.93650794e-02, 7.57811392e+00, 4.00634538e+01, 5.22036697e+00,
       1.66605274e+01, 6.12078522e+00, 2.10897025e+01, 4.91109175e+01,
       1.69587264e+01, 1.39121052e+00, 4.53691612e+00, 1.55243078e+00,
       2.78423800e+01, 4.64529058e-01, 7.27386283e+00, 5.04541293e+00,
       3.16177228e+00, 1.67071269e+01, 6.63315068e+00, 2.86595950e+00,
       6.98232599e-03, 2.49010503e+01, 2.55523955e-01, 5.04001539e+00,
       8.94161677e+00, 2.31712821e+01, 8.38755138e+00, 2.02545877e+01])

In [ ]:
def predict(model, X):
  td = pd.DataFrame(X, columns = features)
  tp = fraction_df(td)
  r = model.predict(tp)
  return r

In [ ]:
a = [3.79414979e+02, 1.37046136e+02, 8.96115747e-01, 5.69255123e+00, 2.01175068e+02, 6.37779876e+02, 1.12112352e+03, 6.03947398e+02]

In [ ]:
predict(rfr_s, [a])

array([51.3409])

In [ ]:
def mass_of_material(M_CaO, M_SiO2, M_K2O, fa=True, s=True):

  if fa==False:
    mass = [M_CaO, M_SiO2]
    Cmax = [np.array([65.59, 48.59]),np.array([19.59, 32.59/100])]   #Cement, Slag
  elif s==False:
    mass = [M_CaO, M_SiO2]
    Cmax = ([np.array([65.59, 1.44])/100, np.array([19.59, 62.75])/100])   #Cement, Flyash
  else:
    mass = [M_CaO, M_SiO2, M_K2O]
    Cmax = [np.array([65.59, 1.44, 48.59])/100 ,np.array([19.59, 62.75, 32.59])/100, np.array([0.57, 2.65, 0.38])/100]   #Cement, Flyash, Slag

  mass_m = np.linalg.solve(Cmax, mass)
  return mass_m

In [ ]:
tdc = 4.9#4.51
tstr = 51#43.2
wc_ratio = 0.35

In [ ]:
from pymoo.core.problem import ElementwiseProblem

class functions(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=n_f,
                         n_obj=2,
                         n_ieq_constr=1,
                         n_eq_constr=0,
                         #'CaO', 'SiO', 'Na2O', 'K2O', 'H20', 'FA', 'CA', 'Total Binder'
                         xl = f_val_min_max(data, features, change_pc=2)[0],
                         xu = f_val_min_max(data, features, change_pc=2)[1]
                         )


    def _evaluate(self, x, out, *args, **kwargs):

        Xt = []

        for i in range(n_f):
          Xt.append(x[i])

        f1 = abs(predict(rfr_d, [Xt]) - tdc)
        f2 = abs(predict(rfr_s, [Xt]) - tstr)

        g1 = x[7] - np.sum(mass_of_material(x[0], x[1], x[3], fa=True, s=True))
        #g2 = -(x[4]/x[7] - 0.99*wc_ratio)
        #g3 = (x[4]/x[7] - 1.01*wc_ratio)

        out["F"] = [f1, f2]
        out["G"] = [g1]#g2, g3]

problem = functions()

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

algorithm = NSGA2(
    pop_size=50,
    n_offsprings=30,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.7, eta=5),
    mutation=PM(eta=5),
    eliminate_duplicates=True
)

from pymoo.termination import get_termination

termination = get_termination("n_gen", 50)

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=rs,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       50 |      3 |  0.000000E+00 |  4.455689E+01 |             - |             -
     2 |       80 |      4 |  0.000000E+00 |  0.000000E+00 |  0.0105736629 |         ideal
     3 |      110 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0564360444 |             f
     4 |      140 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0268724074 |             f
     5 |      170 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0151515152 |         ideal
     6 |      200 |      5 |  0.000000E+00 |  0.000000E+00 |  0.0211628275 |             f
     7 |      230 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0124372721 |         ideal
     8 |      260 |     10 |  0.000000E+00 |  0.000000E+00 |  0.0237008849 |         nadir
     9 |      290 |     13 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |      320 |     17 |  0.000000E+00 |  0.000000E+00 |  0.0025303788 |             f

In [ ]:
X

array([[3.89085282e+02, 1.41216076e+02, 7.04957616e-01, 6.66084893e+00,
        1.94260171e+02, 5.44270929e+02, 9.67606262e+02, 5.93325621e+02],
       [3.88557521e+02, 1.46902127e+02, 6.92487210e-01, 6.66084893e+00,
        1.94439837e+02, 5.50272604e+02, 9.67606262e+02, 5.99640132e+02],
       [3.86431381e+02, 1.46585964e+02, 5.26572209e-01, 6.66084893e+00,
        1.96274468e+02, 5.93786070e+02, 1.07118936e+03, 5.99106129e+02],
       [3.97016393e+02, 1.45078016e+02, 5.93847198e-01, 6.69987311e+00,
        1.93564601e+02, 5.93786070e+02, 1.04496158e+03, 5.99106129e+02],
       [3.94549793e+02, 1.49443416e+02, 7.06138871e-01, 4.63878156e+00,
        1.97233820e+02, 5.44270929e+02, 9.67606262e+02, 5.99105713e+02],
       [3.85935463e+02, 1.44062090e+02, 5.23476852e-01, 8.22074586e+00,
        2.08828179e+02, 6.26946998e+02, 9.91366441e+02, 5.99726009e+02],
       [3.97849748e+02, 1.49438950e+02, 6.60053591e-01, 4.81047191e+00,
        1.94436531e+02, 5.93786070e+02, 1.04464012e+03, 5.

In [ ]:
F

array([[9.1700e-02, 9.2300e-02],
       [1.0390e-01, 7.3500e-02],
       [1.1800e-01, 1.1400e-02],
       [5.1520e-01, 1.2000e-03],
       [2.4730e-01, 1.0400e-02],
       [1.2000e-03, 9.6800e-01],
       [2.5080e-01, 2.2000e-03],
       [1.0390e-01, 7.3500e-02],
       [2.5080e-01, 2.2000e-03],
       [1.1000e-03, 3.4089e+00],
       [1.1000e-03, 3.4089e+00],
       [2.5080e-01, 2.2000e-03],
       [1.7800e-02, 1.3790e-01],
       [5.1520e-01, 1.2000e-03],
       [5.1520e-01, 1.2000e-03],
       [1.6000e-03, 1.5470e-01],
       [1.0390e-01, 7.3500e-02],
       [2.4730e-01, 1.0400e-02],
       [1.7800e-02, 1.3790e-01],
       [1.0330e-01, 7.8300e-02],
       [9.1700e-02, 9.2300e-02]])

In [ ]:
def select(F, X):
  x = [i[0] for i in F]
  y = [i[1] for i in F]
  close_val = abs(45 - np.arctan((np.array(y)/tstr)/(np.array(x)/tdc))*360/np.pi/2)
  close_val = list(close_val)
  indx = close_val.index(min(close_val))
  return X[indx]

In [ ]:
select(F, X)

array([3.85865751e+02, 1.41487057e+02, 9.50272468e-01, 5.85404989e+00,
       1.94346435e+02, 5.99047732e+02, 1.04155302e+03, 5.93325621e+02])